In [2]:
import os

import numpy as np
import pandas as pd
from dotenv import load_dotenv
import tweepy

## Konfiguracja API

Użycie API Twitter 'tweepy' wymaga podania unikalnych deweloperskich tokenów i kluczy, które zostają w naszym przypadku pobrane z pliku .env w katalogu projektu

In [ ]:
load_dotenv()
token = os.getenv('TOKEN')
api_key = os.getenv('API_KEY')
api_key_secret = os.getenv('API_KEY_SECRET')

In [191]:
client = tweepy.Client(
    bearer_token=token,
    consumer_key=api_key,
    consumer_secret=api_key_secret)

In [192]:
tweet_id = 1612201848596832256
username = '@bbudka'
query = f'conversation_id:{tweet_id}'
target_count = 1000

Istnieje ograniczenie dla podstawowej wersji uprawnień do API, które ograniczana pełne scrapowanie do wpisów niestarszych niż 7 dni. W związku z tym, że analizowany przez nas tweet Borysa Budki został opublikowany 8 stycznia 2023, po 15 stycznia 2023 skrypt nie będzie działał zgodnie z zamysłem.

In [193]:
paginator = tweepy.Paginator(
    client.search_recent_tweets,
    query,
    max_results = 100
)

In [194]:
def get_row(tweet: tweepy.Tweet):
    return tweet.text

tweets = pd.DataFrame(
    map(get_row, paginator.flatten(limit=target_count)),
    columns=['tweet']
)

In [195]:
import re

def remove_mentions(text: str):
    pattern = r'\B@\w+'
    return re.sub(pattern, '', text)

def remove_links(text: str):
    pattern = r'http\S+'
    if re.search(pattern, text) is not None:
        return ''
    return text

tweets['tweet'] = tweets['tweet'].apply(remove_mentions).apply(remove_links).apply(str.strip)
tweets.replace('', np.nan, inplace=True)
tweets.dropna(inplace=True)
tweets.reset_index(drop=True, inplace=True)

In [196]:
tweets

,tweet
0,Pięknie pan mówi. W stylu Gebbelsa 😂
1,ciamajdan w Warszawie
2,Gomułka junior zamknij się
3,To wy Budka mówicie że będziecie siłą wyprowad...
4,"Albo Polin, albo Putin - innego wyboru nie ma."
...,...
826,Boże Pan sobie ta wypowiedzią ukręcił bat na w...
827,PO-wski tępak zapomniał już o ciamajdanie itp.
828,A ciamajdan pamiętasz?
829,🤦🤣😂🤣😂🤣. Przecież to wasza strategia.😂🤣😂🤣


In [197]:
df = pd.DataFrame(tweets, columns=['tweet'])
df['sentiment'] = ''
df

,tweet,sentiment
0,Pięknie pan mówi. W stylu Gebbelsa 😂,
1,ciamajdan w Warszawie,
2,Gomułka junior zamknij się,
3,To wy Budka mówicie że będziecie siłą wyprowad...,
4,"Albo Polin, albo Putin - innego wyboru nie ma.",
...,...,...
826,Boże Pan sobie ta wypowiedzią ukręcił bat na w...,
827,PO-wski tępak zapomniał już o ciamajdanie itp.,
828,A ciamajdan pamiętasz?,
829,🤦🤣😂🤣😂🤣. Przecież to wasza strategia.😂🤣😂🤣,


In [198]:
center = len(df) // 2
df1 = df.iloc[:center]
df2 = df.iloc[center:]

In [199]:
df1.to_csv('tweet-replies/budken1.csv', header=True, index=False)
df2.to_csv('tweet-replies/budken2.csv', header=True, index=False)